In [13]:
import pandas as pd 
from gensim.models.word2vec import Word2Vec
import gensim.models.word2vec as word2vec 
from keras.layers import *
from keras.models import Model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.utils import shuffle


In [14]:
data = pd.read_csv('f_clean.csv')


In [15]:
data.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'], inplace = True)

In [16]:
data.loc[data['score'] < 3, 'score'] = -1
data.loc[data['score'] == 3, 'score'] = 0
data.loc[data['score'] > 3, 'score'] = 1
testset = data[:1000000]
testset.reset_index(drop=True)
trainset = data[1000000:]
trainset.reset_index(drop=True)


,text,score
0,loved ergo proxy ever since came back art extr...,1.0
1,one favorite series fans bladerunner ghost she...,1.0
2,absolutely love series watched ever since came...,1.0
3,spoiler free short review ergo proxy brilliant...,1.0
4,kick review like say really much enjoyment wat...,1.0
...,...,...
499091,fan alex loughlin since cbs drama moonlight qu...,1.0
499092,movie reminded classic well written psychologi...,1.0
499093,truly obese women men feed thing deliberately ...,-1.0
499094,really star star originality tried stick one h...,-1.0


In [18]:
x_train = list(trainset.text)
x_test = list(testset.text)

In [19]:
import numpy as np 
y_train = np.array(trainset.score)
y_test = np.array(testset.score)

In [21]:
max_feature = 5000
maxlen = 50
embed_size = 25

In [23]:
tokenizer = Tokenizer(num_words=max_feature)
tokenizer.fit_on_texts(x_train)
token_train = tokenizer.texts_to_sequences(x_train)
token_test = tokenizer.texts_to_sequences(x_test)

In [24]:
x_train = pad_sequences(token_train, maxlen=maxlen, padding='post')
x_test = pad_sequences(token_test, maxlen=maxlen, padding='post')

In [26]:
from IPython.display import clear_output
import keras
import matplotlib.pyplot as plt

class PlotLosses(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        
        self.fig = plt.figure()
        
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.i += 1
        
        clear_output(wait=True)
        plt.plot(self.x, self.losses, label="loss")
        plt.plot(self.x, self.val_losses, label="val_loss")
        plt.legend()
        plt.show();
        
plot_losses = PlotLosses()

In [28]:
from keras import optimizers
from keras import regularizers
from keras import Sequential
eta = 1

In [31]:
hidden_size = 50

In [32]:
model = Sequential()
model.add(Embedding(max_feature, embed_size, input_length=maxlen))
model.add(LSTM(hidden_size, dropout=0.2, recurrent_dropout=0.3))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [33]:
batch_size = 32
num_epoch = 10
model.fit(x_train,y_train,batch_size=batch_size,epochs=num_epoch)

Epoch 1/10
  597/15597 [>.............................] - ETA: 1:44:54 - loss: 0.4722 - accuracy: 0.7240